# Frozen Lake 

https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py

https://medium.com/swlh/introduction-to-reinforcement-learning-coding-q-learning-part-3-9778366a41c0

https://www.analyticsvidhya.com/blog/2018/09/reinforcement-learning-model-based-planning-dynamic-programming/

### What's in this notebook?

I explore three different methods for solving the frozen lake problem. The methods are: 1) the Q-learn method, 2) the policy iteration method, and 3) the value iteration method. 

From what I can tell, the Q-learn method requires no prior knowledge of the environment, however, method (2,3) require complete knowledge of the environment.

### Desription of Frozen Lake Problem

S is the starting point, G is the goal, F is the solid ice where the agent can stand and H is the hole where if the agent goes, it falls down.

For every state F, S, the agent gets 0 reward. In state H, the agent will die and upon reaching G, the agent gets +1 reward.


The agent in the environment has four possible moves — Up, Down, Left and Right. We will be implementing one of the Reinforcement Learning techniques, Q-Learning, here. This environment will allow the agent to move accordingly. There could be a random action happening once every a few episodes — let’s say the agent is slipping in different directions because it is hard to walk on a frozen surface. Considering this situation, we need to allow some random movement at first, but eventually try to reduce its probability. This way we can correct the error caused by minimising the loss.

Through some fiddling, it was found that if action = 0 (left), there is a 0.33 probability that you will go either up, left or down. This pattern is the same for the other 3 actions.

In [ ]:
import gym
import numpy as np
import time, pickle, os
import matplotlib.pyplot as plt

In [99]:
# A quick run of the CartPole enviroment with random actions
# prints out the observations
env = gym.make("FrozenLake-v0")
for i_episode in range(10):
    observation = env.reset()
    for t in range(50):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()


SFFF
FHFH
FFFH
HFFG
0
  (Right)
SFFF
FHFH
FFFH
HFFG
1
  (Left)
SFFF
FHFH
FFFH
HFFG
1
  (Left)
SFFF
FHFH
FFFH
HFFG
0
  (Right)
SFFF
FHFH
FFFH
HFFG
0
  (Down)
SFFF
FHFH
FFFH
HFFG
0
  (Up)
SFFF
FHFH
FFFH
HFFG
0
  (Down)
SFFF
FHFH
FFFH
HFFG
4
  (Up)
SFFF
FHFH
FFFH
HFFG
4
  (Down)
SFFF
FHFH
FFFH
HFFG
4
  (Left)
SFFF
FHFH
FFFH
HFFG
8
  (Right)
SFFF
FHFH
FFFH
HFFG
4
  (Right)
SFFF
FHFH
FFFH
HFFG
0
  (Up)
SFFF
FHFH
FFFH
HFFG
0
  (Right)
SFFF
FHFH
FFFH
HFFG
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
  (Up)
SFFF
FHFH
FFFH
HFFG
1
  (Up)
SFFF
FHFH
FFFH
HFFG
2
  (Up)
SFFF
FHFH
FFFH
HFFG
3
  (Down)
SFFF
FHFH
FFFH
HFFG
3
  (Down)
SFFF
FHFH
FFFH
HFFG
3
  (Right)
SFFF
FHFH
FFFH
HFFG
3
Episode finished after 23 timesteps

SFFF
FHFH
FFFH
HFFG
0
  (Left)
SFFF
FHFH
FFFH
HFFG
0
  (Up)
SFFF
FHFH
FFFH
HFFG
0
  (Right)
SFFF
FHFH
FFFH
HFFG
4
  (Down)
SFFF
FHFH
FFFH
HFFG
4
  (Right)
SFFF
FHFH
FFFH
HFFG
0
  (Down)
SFFF
FHFH
FFFH
HFFG
1
  (Up)
SFFF
FHFH
FFFH
HFFG
0
  (Right)
SFFF
FHFH
FFFH
HF

## Solving the problem

### Q - Learning Method

Let's start off by importing some stuff and initialize the frozen lake environment. 

In [77]:
import gym
import numpy as np
import time, pickle, os

env = gym.make('FrozenLake-v0')

#### Initialize Variables

Lines 7–12 initializes our variables. epsilon for the epsilon-greedy approach, gamma is the discount factor, max_episodes is the maximum amount of times we’ll run the game, max_steps is the maximum steps we’ll run for every episode and lr_rate is the learning rate.

Line 14 initializes our Q-table as a 16x4 matrix filled with zeros. env.observation-space.n tells the total number of states in the game and env.action-space.n tells the total number of actions.

In [94]:
epsilon = 0.5 # should lower this once the model is trained
total_episodes = 10000
max_steps = 100

lr_rate = 0.81
gamma = 0.96

# Q = np.zeros((env.observation_space.n, env.action_space.n))
# or load it from a saved run
Q = pickle.load( open( "frozenLake_qTable.pkl", "rb" ) )
Q

array([[0.67697129, 0.58169564, 0.578977  , 0.63370152],
       [0.4891982 , 0.10332174, 0.09701621, 0.60267096],
       [0.60113092, 0.59147936, 0.61121603, 0.60895209],
       [0.11222578, 0.11464126, 0.59782599, 0.59773137],
       [0.71036386, 0.13285549, 0.12607732, 0.0225453 ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.63249179, 0.02198047, 0.13561132, 0.53627441],
       [0.        , 0.        , 0.        , 0.        ],
       [0.64288817, 0.68961708, 0.71578025, 0.82091172],
       [0.64322003, 0.77698643, 0.81852143, 0.73170506],
       [0.78284985, 0.13861668, 0.02394721, 0.60005945],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.82688113, 0.17551073, 0.86018317, 0.77479256],
       [0.84234249, 0.9812486 , 0.91540066, 0.85459699],
       [0.        , 0.        , 0.        , 0.        ]])

In [95]:
def choose_action(state):
    action=0
    if np.random.uniform(0, 1) < epsilon:
        action = env.action_space.sample()
        # print("Action random: ", action)
    else:
        action = np.argmax(Q[state, :])
        # print("Action argmax: ", action)
    return action

def learn(state, state2, reward, action):
    predict = Q[state, action]
    target = reward + gamma * np.max(Q[state2, :])
    Q[state, action] = Q[state, action] + lr_rate * (target - predict)

**State** - Goes from $0-15$ where $0$ is S and $15$ is G.

**Q-table** - Rows are states, columns are actions. Therefore, 16 states and 4 actions.

**Action** - 0, 1, 2, 3 -> left, down, right, up

In [98]:
# Start
tempval = 0
temp_ep = []
wins = 0
for episode in range(total_episodes):
    
#     print("\nEpisode: ", episode)
    state = env.reset()
    t = 0
    
    while t < max_steps:
        # env.render()

        action = choose_action(state) # select action
        state2, reward, done, info = env.step(action) # given action, return enviroment variables 
        learn(state, state2, reward, action) # update Q table with results
        state = state2 # state(n+1) = state(n)

        t += 1
        
        if done and reward == 1.0: # if the goal is reached
            wins += 1
            # print("----------------------------------")
            # print("MADE IT TO THE GOAL!!!")
            # print("----------------------------------")
            temp_ep.append(episode)
            # print(Q)
        
        if done: # either at the goal or in a hole
            break

        # time.sleep(0.1)

# Check how many episodes were successful
print(f'Q-Learn :: number of wins over {total_episodes} episodes = {wins}')
    
# print(Q)

# Print the percentage of successful runs
percentage_solved = len(temp_ep)/total_episodes*100
print("\nPercentage Solved = ", percentage_solved, "%")

# save the learned Q matrix
with open("frozenLake_qTable.pkl", 'wb') as f:
    pickle.dump(Q, f)

Q-Learn :: number of wins over 10000 episodes = 321

Percentage Solved =  3.2099999999999995 %


### Policy Iteration in python

https://www.analyticsvidhya.com/blog/2018/09/reinforcement-learning-model-based-planning-dynamic-programming/

Let’s start with the policy evaluation step. The objective is to converge to the true value function for a given policy π. We will define a function that returns the required value function.

**The value function** - The expected value of a state given an action for the state (v)/ given all the actions for the state (V).  

In [1]:
def policy_evaluation(policy, environment, discount_factor=1.0, theta=1e-9, max_iterations=1e9):
        # Number of evaluation iterations
        evaluation_iterations = 1
        # Initialize a value function for each state as zero
        V = np.zeros(environment.nS)
        # Repeat until change in value is below the threshold
        for i in range(int(max_iterations)):
            # Initialize a change of value function as zero
            delta = 0
            # Iterate though each state
            for state in range(environment.nS):
               # Initial a new value of current state
               v = 0
               # Try all possible actions which can be taken from this state
               for action, action_probability in enumerate(policy[state]):
                    # Check how good next state will be
                    for state_probability, next_state, reward, terminated in environment.P[state][action]:
                        # Calculate the expected value
                        v += action_probability * state_probability * (reward + discount_factor * V[next_state])

               # Calculate the absolute change of value function
               delta = max(delta, np.abs(V[state] - v))
               # Update value function
               V[state] = v
            evaluation_iterations += 1

            # Terminate if value change is insignificant
            if delta < theta:
                    print(f'Policy evaluated in {evaluation_iterations} iterations.')
                    return V

Now coming to the policy improvement part of the policy iteration algorithm. We need a helper function that does one step lookahead to calculate the state-value function. This will return an array of length nA containing expected value of each action

In [2]:
def one_step_lookahead(environment, state, V, discount_factor):
        action_values = np.zeros(environment.nA)
        for action in range(environment.nA):
                for probability, next_state, reward, terminated in environment.P[state][action]:
                        action_values[action] += probability * (reward + discount_factor * V[next_state])
        return action_values

Now, the overall policy iteration would be as described below. This will return a tuple (policy,V) which is the optimal policy matrix and value function for each state.

In [62]:
def policy_iteration(environment, discount_factor=1.0, max_iterations=1e9):
        # Start with a random policy
        # (num states by num actions) / num actions
        policy = np.ones([environment.nS, environment.nA]) / environment.nA 
        # Initialize counter of evaluated policies
        evaluated_policies = 1
        # Repeat until convergence or critical number of iterations reached
        for i in range(int(max_iterations)):
            stable_policy = True
            # Evaluate current policy
            V = policy_evaluation(policy, environment, discount_factor=discount_factor)
            # Go through each state and try to improve actions that were taken (policy Improvement)
            for state in range(environment.nS):
                # Choose the best action in a current state under current policy
                current_action = np.argmax(policy[state])
                # Look one step ahead and evaluate if current action is optimal
                # We will try every possible action in a current state
                action_value = one_step_lookahead(environment, state, V, discount_factor)
                # Select a better action
                best_action = np.argmax(action_value)
                # If action didn't change
                if current_action != best_action: # I made a change here
                    stable_policy = False
                    # Greedy policy update
                    policy[state] = np.eye(environment.nA)[best_action]
                else:
                    policy[state] = np.eye(environment.nA)[best_action]
            evaluated_policies += 1
            # If the algorithm converged and policy is not changing anymore, then return final policy and value function
            if stable_policy:
                print(f'Evaluated {evaluated_policies} policies.')
                return policy, V

### Value Iteration in python

The parameters are defined in the same manner for value iteration. The value iteration algorithm can be similarly coded:

In [70]:
def value_iteration(environment, discount_factor=1.0, theta=1e-9, max_iterations=1e9):
        # Initialize state-value function with zeros for each environment state
        V = np.zeros(environment.nS)
        for i in range(int(max_iterations)):
            # Early stopping condition
            delta = 0
            # Update each state
            for state in range(environment.nS):
                    # Do a one-step lookahead to calculate state-action values
                    action_value = one_step_lookahead(environment, state, V, discount_factor)
                    # Select best action to perform based on the highest state-action value
                    best_action_value = np.max(action_value)
                    # Calculate change in value
                    delta = max(delta, np.abs(V[state] - best_action_value))
                    # Update the value function for current state
                    V[state] = best_action_value
                    # Check if we can stop
            if delta < theta:
                    print(f'Value-iteration converged at iteration#{i}.')
                    break

        # Create a deterministic policy using the optimal value function
        policy = np.zeros([environment.nS, environment.nA])
        for state in range(environment.nS):
            # One step lookahead to find the best action for this state
            action_value = one_step_lookahead(environment, state, V, discount_factor)
            # Select best action based on the highest state-action value
            best_action = np.argmax(action_value)
            # Update the policy to perform a better action at a current state
            policy[state, best_action] = 1.0
        return policy, V

### Compare the methods

Finally, let’s compare both methods to look at which of them works better in a practical setting. To do this, we will try to learn the optimal policy for the frozen lake environment using both techniques described above. Later, we will check which technique performed better based on the average return after 10,000 episodes.

In [72]:
def play_episodes(environment, n_episodes, policy):
    wins = 0
    total_reward = 0
    for episode in range(n_episodes):
        terminated = False
        state = environment.reset()
        while not terminated:
            # Select best action to perform in a current state
            action = np.argmax(policy[state])
            # Perform an action an observe how environment acted in response
            next_state, reward, terminated, info = environment.step(action)
            # Summarize total reward
            total_reward += reward
            # Update current state
            state = next_state
            # Calculate number of wins over episodes
            if terminated and reward == 1.0:
                wins += 1
    average_reward = total_reward / n_episodes
    return wins, total_reward, average_reward

# Number of episodes to play
n_episodes = 10000
# Functions to find best policy
solvers = [('Policy Iteration', policy_iteration),
           ('Value Iteration', value_iteration)]
for iteration_name, iteration_func in solvers:
    # Load a Frozen Lake environment
    environment = gym.make('FrozenLake-v0')
    # Search for an optimal policy using policy iteration
    policy, V = iteration_func(environment.env)
    # Apply best policy to the real environment
    wins, total_reward, average_reward = play_episodes(environment, n_episodes, policy)
    print(f'{iteration_name} :: number of wins over {n_episodes} episodes = {wins}')
    print(f'{iteration_name} :: average reward over {n_episodes} episodes = {average_reward} \n\n')

Policy evaluated in 66 iterations.
Policy evaluated in 417 iterations.
Policy evaluated in 527 iterations.
Evaluated 4 policies.
Policy Iteration :: number of wins over 10000 episodes = 7371
Policy Iteration :: average reward over 10000 episodes = 0.7371 


Value-iteration converged at iteration#523.
Value Iteration :: number of wins over 10000 episodes = 7416
Value Iteration :: average reward over 10000 episodes = 0.7416 




### Q-Learning Using Tensor Flow - (Not working yet)

https://gist.github.com/analyticsindiamagazine/381977a5831835b4c0dddbb53e6c1b70

We will use Gradient optimizer to reduce the loss with a learning rate of 0.1

In [9]:
import tensorflow as tf

In [8]:
Q2 = tf.placeholder(shape=[1,4],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(Q2 - Q1))
gdo = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updatedweights = gdo.minimize(loss)

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [ ]:
gamma = 0.9
epsilon = 0.1
episodes = 2000

totalReward = 0

session = tf.Session()
session.run(tf.initialize_all_variables())
for i in range(episodes):
    state_now = env.reset()
    done = False
    reward = 0
    for j in range(100):
        #Lets find the dot product of the inputs with the weights and apply argmax on it.
        action , Y = session.run([output, Q1], feed_dict = {inputs : [np.eye(16)[state_now]]})
        if epsilon > np.random.rand(1):
            action[0] = env.action_space.sample()
            epsilon -= 10**-3
        #Lets iterate to the next state Note: This can be random.
        state_next , reward, done, _ = env.step(action[0])
        Y1 = session.run(Q1, feed_dict = {inputs : [np.eye(16)[state_next]]})
        change_Y = Y
        change_Y[0, action[0]] = reward + gamma*np.max(Y1)
        #Updating the weights 
        _,new_weights = session.run([updatedweights,weights],feed_dict={inputs:[np.eye(16)[state_now]],Q2:change_Y})
        #Lets append the total number of rewards
        totalReward += reward
        state_now = state_next
        if reward == 1:
            print ('Episode {} was successful, Agent reached the Goal'.format(i))
            
session.close()